# 02 — Exploratory Data Analysis: Environmental & Anthropogenic Drivers of Shark Dynamics

**Objective**: Characterize spatial and temporal patterns of key drivers affecting shark populations along the Brazilian coast.

**Data sources**:
1. **Sea Surface Temperature (SST)** — NOAA OISST v2.1 (2010-2023, monthly)
2. **Species occurrences** — OBIS (sharks: *Carcharhinus longimanus*, *Galeocerdo cuvier*; prey: 7 families)
3. **Fishing effort** — Global Fishing Watch (2020-2024, monthly)

**Region**: Brazilian Exclusive Economic Zone (-35°S to 5°N, -50°W to -30°W)

**Resolution**: 1° × 1° grid cells (monthly aggregation)

In [1]:
# Imports e config
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
from tqdm.auto import tqdm

from nova_selachiia.config import ProjectConfig, RegionBBox, as_tuple

# Carregar config do projeto
cfg = ProjectConfig(
    region=RegionBBox(lat_min=-35.0, lat_max=5.0, lon_min=-50.0, lon_max=-30.0),
    grid_deg=1.0,
    shark_species=as_tuple([
        "Prionace glauca",
        "Carcharhinus longimanus",
        "Sphyrna lewini",
        "Isurus oxyrinchus",
        "Carcharhinus leucas",
        "Galeocerdo cuvier",
    ]),
    prey_groups=as_tuple([
        "Clupeidae",
        "Engraulidae",
        "Scombridae",
        "Carangidae",
        "Loliginidae",
        "Octopodidae",
        "Trichiuridae",
        "Mugilidae",
    ]),
    collapse_threshold_quantile=0.10,
    mc_trajectories=200,
    random_seed=42,
    delta_scenarios=(-0.2, -0.1, 0.0, 0.1, 0.2),
)

DATA_DIR = Path("../data")
DATA_DIR.mkdir(exist_ok=True)

print(f"Região: {cfg.region}")
print(f"Grid: {cfg.grid_deg}°")
print(f"Tubarões: {len(cfg.shark_species)} espécies")
print(f"Presas: {len(cfg.prey_groups)} grupos")

Região: RegionBBox(lat_min=-35.0, lat_max=5.0, lon_min=-50.0, lon_max=-30.0)
Grid: 1.0°
Tubarões: 6 espécies
Presas: 8 grupos


c:\Users\rugge_p2gkz2r\AppData\Local\pypoetry\Cache\virtualenvs\nova-selachiia-oFZsEAWQ-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1) Grid espacial e máscara

Definir células 1°×1° na região de interesse (costa brasileira).

In [2]:
def make_grid(region: RegionBBox, resolution: float) -> pd.DataFrame:
    """Cria grid de células com centro lat/lon."""
    lats = np.arange(region.lat_min + resolution / 2, region.lat_max, resolution)
    lons = np.arange(region.lon_min + resolution / 2, region.lon_max, resolution)
    
    cells = []
    for lat in lats:
        for lon in lons:
            cells.append({
                "lat": lat,
                "lon": lon,
                "lat_min": lat - resolution / 2,
                "lat_max": lat + resolution / 2,
                "lon_min": lon - resolution / 2,
                "lon_max": lon + resolution / 2,
            })
    return pd.DataFrame(cells)

grid = make_grid(cfg.region, cfg.grid_deg)
print(f"Total de células no grid: {len(grid)}")
grid.head()

Total de células no grid: 800


,lat,lon,lat_min,lat_max,lon_min,lon_max
0,-34.5,-49.5,-35.0,-34.0,-50.0,-49.0
1,-34.5,-48.5,-35.0,-34.0,-49.0,-48.0
2,-34.5,-47.5,-35.0,-34.0,-48.0,-47.0
3,-34.5,-46.5,-35.0,-34.0,-47.0,-46.0
4,-34.5,-45.5,-35.0,-34.0,-46.0,-45.0


## 🦈 Estratégia para Hardware Limitado (8GB RAM, sem GPU)

**Problema**: 249k registros de Carcharhiniformes seria pesado demais.

**Solução**: Trabalhar apenas com os dados **já baixados** (esparsos mas viáveis):
- 2 espécies de tubarão: C. longimanus (7 reg), G. cuvier (19 reg)
- 7 famílias de presas: volumes maiores

**Otimizações**:
1. Agregação mensal por célula de grid (reduz dimensionalidade)
2. Usar apenas 2018-2023 (5 anos em vez de 13)
3. Grid 1°×1° (já é baixa resolução)
4. Processar apenas células com dados (ignora oceano aberto)

In [ ]:
# ============================================================================
# DATA INVENTORY
# ============================================================================
print("═" * 80)
print("DATA INVENTORY — Brazilian Coast (2010-2024)")
print("═" * 80)

raw_dir = DATA_DIR / "raw"

# Tubarões
shark_files = sorted(raw_dir.glob("shark_*.csv"))
shark_data = {}
print("\n🦈 SHARKS (Elasmobranchii)")
print("-" * 80)
for f in shark_files:
    df_temp = pd.read_csv(f)
    species = f.stem.replace("shark_", "").replace("_", " ").title()
    shark_data[species] = df_temp
    
    # Parse dates
    if 'eventDate' in df_temp.columns:
        dates = pd.to_datetime(df_temp['eventDate'], errors='coerce')
        date_range = f"{dates.min().year}-{dates.max().year}" if dates.notna().any() else "N/A"
    else:
        date_range = "N/A"
    
    print(f"  {species:35s} {len(df_temp):6,} records   {date_range}")

# Presas
print(f"\n🐟 PREY TAXA (Teleostei & Cephalopoda)")
print("-" * 80)
prey_files = sorted(raw_dir.glob("prey_*.csv"))
prey_data = {}
for f in prey_files:
    df_temp = pd.read_csv(f)
    family = f.stem.replace("prey_", "")
    prey_data[family] = df_temp
    
    if 'eventDate' in df_temp.columns:
        dates = pd.to_datetime(df_temp['eventDate'], errors='coerce')
        date_range = f"{dates.min().year}-{dates.max().year}" if dates.notna().any() else "N/A"
    else:
        date_range = "N/A"
    
    print(f"  {family:35s} {len(df_temp):6,} records   {date_range}")

# SST
print(f"\n🌡️  SEA SURFACE TEMPERATURE")
print("-" * 80)
sst_file = raw_dir / "sst.mnmean.nc"
if sst_file.exists():
    size_mb = sst_file.stat().st_size / 1024 / 1024
    print(f"  NOAA OISST v2.1 monthly         {size_mb:6.1f} MB")
    
    ds_info = xr.open_dataset(sst_file)
    time_range = f"{str(ds_info.time.values[0])[:10]} to {str(ds_info.time.values[-1])[:10]}"
    print(f"  Temporal coverage:              {time_range}")
    print(f"  Timesteps:                      {len(ds_info.time):6,}")
    print(f"  Spatial resolution:             0.25° (~25 km)")
    ds_info.close()
else:
    print("  ⚠️  File not found: sst.mnmean.nc")

# Fishing
print(f"\n🎣 FISHING EFFORT (Global Fishing Watch)")
print("-" * 80)
fleet_dirs = sorted(raw_dir.glob("fleet-monthly-*"))
print(f"  Years available:                {len(fleet_dirs)} ({', '.join([d.name[-4:] for d in fleet_dirs])})")
if fleet_dirs:
    all_csvs = list(fleet_dirs[0].glob("*.csv"))
    if all_csvs:
        df_sample = pd.read_csv(all_csvs[0])
        print(f"  Records per month (example):    {len(df_sample):6,}")
        print(f"  Variables:                      {', '.join(df_sample.columns[:5])}...")

print("\n" + "═" * 80)
print(f"💾 MEMORY ESTIMATE: < 500 MB (viable for 8GB RAM)")
print("═" * 80)

🦈 TUBARÕES
------------------------------------------------------------
  Carcharhinus leucas            →    17 registros
  Carcharhinus longimanus        →     7 registros
  Galeocerdo cuvier              →    19 registros

🐟 PRESAS
------------------------------------------------------------
  Carangidae                     → 1,384 registros
  Clupeidae                      →    26 registros
  Engraulidae                    →    65 registros
  Mugilidae                      →   110 registros
  Octopodidae                    →     9 registros
  Scombridae                     →    37 registros
  Trichiuridae                   →     1 registros

🎣 FISHING EFFORT
------------------------------------------------------------
  5 diretórios (anos 2020-2024)
  Exemplo (1 mês): 994,117 registros
  Colunas: ['date', 'year', 'month', 'cell_ll_lat', 'cell_ll_lon', 'flag', 'geartype', 'hours', 'fishing_hours', 'mmsi_present']

🌡️ SST
------------------------------------------------------------
 

## 1. Spatial Coverage Analysis

Characterize geographic distribution of observations and identify data gaps.

In [ ]:
# ============================================================================
# SPATIAL DISTRIBUTION OF OCCURRENCES
# ============================================================================
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec

# Set publication-ready style
plt.style.use('seaborn-v0_8-paper')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 150
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12

# Combine all occurrence data
all_sharks = []
for species, df in shark_data.items():
    if 'decimalLatitude' in df.columns:
        temp = df[['decimalLatitude', 'decimalLongitude']].copy()
        temp['taxon'] = species
        temp['group'] = 'Shark'
        all_sharks.append(temp)

all_prey = []
for family, df in prey_data.items():
    if 'decimalLatitude' in df.columns:
        temp = df[['decimalLatitude', 'decimalLongitude']].copy()
        temp['taxon'] = family
        temp['group'] = 'Prey'
        all_prey.append(temp)

if all_sharks:
    shark_occ = pd.concat(all_sharks, ignore_index=True)
    shark_occ.columns = ['lat', 'lon', 'taxon', 'group']
else:
    shark_occ = pd.DataFrame(columns=['lat', 'lon', 'taxon', 'group'])

if all_prey:
    prey_occ = pd.concat(all_prey, ignore_index=True)
    prey_occ.columns = ['lat', 'lon', 'taxon', 'group']
else:
    prey_occ = pd.DataFrame(columns=['lat', 'lon', 'taxon', 'group'])

all_occ = pd.concat([shark_occ, prey_occ], ignore_index=True)

# Spatial coverage map
fig = plt.figure(figsize=(14, 10))
gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.3)

# Panel A: All occurrences
ax1 = fig.add_subplot(gs[0, :])
if len(shark_occ) > 0:
    ax1.scatter(shark_occ['lon'], shark_occ['lat'], 
                c='#e74c3c', s=30, alpha=0.6, label='Sharks', edgecolors='k', linewidth=0.3)
if len(prey_occ) > 0:
    ax1.scatter(prey_occ['lon'], prey_occ['lat'], 
                c='#3498db', s=20, alpha=0.4, label='Prey taxa', edgecolors='none')

ax1.set_xlabel('Longitude (°W)', fontweight='bold')
ax1.set_ylabel('Latitude (°S/°N)', fontweight='bold')
ax1.set_title('A) Geographic Distribution of OBIS Records (2010-2023)', 
              fontweight='bold', loc='left', pad=10)
ax1.legend(frameon=True, loc='upper right')
ax1.grid(True, alpha=0.3, linestyle='--')
ax1.set_xlim(cfg.region.lon_min, cfg.region.lon_max)
ax1.set_ylim(cfg.region.lat_min, cfg.region.lat_max)

# Panel B: Shark species breakdown
ax2 = fig.add_subplot(gs[1, 0])
if len(shark_occ) > 0:
    shark_counts = shark_occ['taxon'].value_counts()
    colors = sns.color_palette("Reds_r", len(shark_counts))
    bars = ax2.barh(range(len(shark_counts)), shark_counts.values, color=colors, edgecolor='k', linewidth=0.5)
    ax2.set_yticks(range(len(shark_counts)))
    ax2.set_yticklabels([s.replace('Carcharhinus ', 'C. ').replace('Galeocerdo ', 'G. ') 
                          for s in shark_counts.index], style='italic')
    ax2.set_xlabel('Number of records', fontweight='bold')
    ax2.set_title('B) Shark Species Coverage', fontweight='bold', loc='left', pad=10)
    ax2.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Add counts on bars
    for i, (bar, val) in enumerate(zip(bars, shark_counts.values)):
        ax2.text(val + 0.5, i, f'{val}', va='center', fontsize=9)
else:
    ax2.text(0.5, 0.5, 'No shark data available', ha='center', va='center', transform=ax2.transAxes)
    ax2.set_xlim(0, 1)
    ax2.set_ylim(0, 1)

# Panel C: Prey family breakdown
ax3 = fig.add_subplot(gs[1, 1])
if len(prey_occ) > 0:
    prey_counts = prey_occ['taxon'].value_counts()
    colors = sns.color_palette("Blues_r", len(prey_counts))
    bars = ax3.barh(range(len(prey_counts)), prey_counts.values, color=colors, edgecolor='k', linewidth=0.5)
    ax3.set_yticks(range(len(prey_counts)))
    ax3.set_yticklabels(prey_counts.index, style='italic')
    ax3.set_xlabel('Number of records', fontweight='bold')
    ax3.set_title('C) Prey Taxa Coverage', fontweight='bold', loc='left', pad=10)
    ax3.grid(axis='x', alpha=0.3, linestyle='--')
    
    # Add counts on bars
    for i, (bar, val) in enumerate(zip(bars, prey_counts.values)):
        ax3.text(val + max(prey_counts)*0.02, i, f'{val:,}', va='center', fontsize=9)
else:
    ax3.text(0.5, 0.5, 'No prey data available', ha='center', va='center', transform=ax3.transAxes)
    ax3.set_xlim(0, 1)
    ax3.set_ylim(0, 1)

plt.savefig(DATA_DIR / 'figures' / 'fig1_spatial_coverage.png', dpi=300, bbox_inches='tight')
plt.savefig(DATA_DIR / 'figures' / 'fig1_spatial_coverage.pdf', bbox_inches='tight')
print(f"\n✅ Figure saved: fig1_spatial_coverage.png/pdf")

# Create figures directory
(DATA_DIR / 'figures').mkdir(exist_ok=True)

## 2. Temporal Patterns: Sea Surface Temperature

Characterize long-term trends, seasonality, and spatial gradients in SST.

In [ ]:
# ============================================================================
# LOAD AND PROCESS SST DATA
# ============================================================================
print("Loading SST data from local NetCDF...")

ds_sst = xr.open_dataset(raw_dir / "sst.mnmean.nc")

# Subset to region of interest
sst_subset = ds_sst.sel(
    lat=slice(cfg.region.lat_min, cfg.region.lat_max),
    lon=slice(cfg.region.lon_min + 360, cfg.region.lon_max + 360)
)

# Convert to DataFrame
sst_df = sst_subset['sst'].to_dataframe().reset_index()
sst_df['lon'] = sst_df['lon'] - 360  # Convert to [-180, 180]
sst_df = sst_df.dropna(subset=['sst'])

# Filter to 2010-2023
sst_df['time'] = pd.to_datetime(sst_df['time'])
sst_df = sst_df[(sst_df['time'].dt.year >= 2010) & (sst_df['time'].dt.year <= 2023)]

print(f"✅ SST data loaded: {len(sst_df):,} observations")
print(f"   Period: {sst_df['time'].min().date()} to {sst_df['time'].max().date()}")
print(f"   Temperature range: {sst_df['sst'].min():.1f}°C to {sst_df['sst'].max():.1f}°C")

# Calculate monthly regional means
sst_ts = sst_df.groupby(sst_df['time'].dt.to_period('M')).agg({
    'sst': ['mean', 'std', 'min', 'max']
}).reset_index()
sst_ts.columns = ['month', 'sst_mean', 'sst_std', 'sst_min', 'sst_max']
sst_ts['time'] = sst_ts['month'].dt.to_timestamp()
sst_ts['year'] = sst_ts['time'].dt.year
sst_ts['month_num'] = sst_ts['time'].dt.month

# Calculate latitudinal gradient
sst_lat = sst_df.groupby('lat').agg({
    'sst': ['mean', 'std']
}).reset_index()
sst_lat.columns = ['lat', 'sst_mean', 'sst_std']

print(f"\n📊 Regional statistics:")
print(f"   Mean SST: {sst_ts['sst_mean'].mean():.2f} ± {sst_ts['sst_std'].mean():.2f}°C")
print(f"   Seasonal amplitude: {sst_ts['sst_mean'].max() - sst_ts['sst_mean'].min():.2f}°C")

In [ ]:
# ============================================================================
# SST VISUALIZATION: Time series, seasonality, spatial patterns
# ============================================================================
from scipy import stats

fig = plt.figure(figsize=(16, 10))
gs = GridSpec(3, 3, figure=fig, hspace=0.35, wspace=0.35)

# Panel A: Long-term time series with trend
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(sst_ts['time'], sst_ts['sst_mean'], linewidth=1.5, color='#2c3e50', label='Monthly mean')
ax1.fill_between(sst_ts['time'], 
                 sst_ts['sst_mean'] - sst_ts['sst_std'],
                 sst_ts['sst_mean'] + sst_ts['sst_std'],
                 alpha=0.2, color='#3498db', label='±1 SD')

# Linear trend
x_numeric = (sst_ts['time'] - sst_ts['time'].min()).dt.days.values
slope, intercept, r_value, p_value, std_err = stats.linregress(x_numeric, sst_ts['sst_mean'])
trend_line = slope * x_numeric + intercept
ax1.plot(sst_ts['time'], trend_line, '--', color='#e74c3c', linewidth=2, 
         label=f'Trend: {slope*365:.3f}°C/year (p={p_value:.3f})')

ax1.set_xlabel('Year', fontweight='bold')
ax1.set_ylabel('SST (°C)', fontweight='bold')
ax1.set_title('A) Sea Surface Temperature Time Series (2010-2023)', fontweight='bold', loc='left', pad=10)
ax1.legend(frameon=True, loc='upper left')
ax1.grid(True, alpha=0.3, linestyle='--')

# Panel B: Seasonal climatology
ax2 = fig.add_subplot(gs[1, 0])
monthly_clim = sst_ts.groupby('month_num').agg({
    'sst_mean': ['mean', 'std']
}).reset_index()
monthly_clim.columns = ['month', 'sst_mean', 'sst_std']

months_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
ax2.plot(monthly_clim['month'], monthly_clim['sst_mean'], 'o-', 
         linewidth=2, markersize=8, color='#e74c3c')
ax2.fill_between(monthly_clim['month'],
                 monthly_clim['sst_mean'] - monthly_clim['sst_std'],
                 monthly_clim['sst_mean'] + monthly_clim['sst_std'],
                 alpha=0.3, color='#e74c3c')
ax2.set_xlabel('Month', fontweight='bold')
ax2.set_ylabel('SST (°C)', fontweight='bold')
ax2.set_title('B) Seasonal Climatology', fontweight='bold', loc='left', pad=10)
ax2.set_xticks(range(1, 13))
ax2.set_xticklabels(months_labels, rotation=45)
ax2.grid(True, alpha=0.3, linestyle='--')

# Panel C: Latitudinal gradient
ax3 = fig.add_subplot(gs[1, 1])
ax3.plot(sst_lat['sst_mean'], sst_lat['lat'], 'o-', linewidth=2, 
         markersize=6, color='#3498db')
ax3.fill_betweenx(sst_lat['lat'],
                  sst_lat['sst_mean'] - sst_lat['sst_std'],
                  sst_lat['sst_mean'] + sst_lat['sst_std'],
                  alpha=0.3, color='#3498db')
ax3.set_ylabel('Latitude (°)', fontweight='bold')
ax3.set_xlabel('SST (°C)', fontweight='bold')
ax3.set_title('C) Latitudinal Gradient', fontweight='bold', loc='left', pad=10)
ax3.grid(True, alpha=0.3, linestyle='--')
ax3.axhline(0, color='k', linestyle='--', linewidth=0.8, alpha=0.5)

# Panel D: Interannual variability
ax4 = fig.add_subplot(gs[1, 2])
annual_means = sst_ts.groupby('year')['sst_mean'].mean().reset_index()
colors_yearly = plt.cm.RdYlBu_r((annual_means['sst_mean'] - annual_means['sst_mean'].min()) / 
                                (annual_means['sst_mean'].max() - annual_means['sst_mean'].min()))
bars = ax4.bar(annual_means['year'], annual_means['sst_mean'], color=colors_yearly, edgecolor='k', linewidth=0.5)
ax4.set_xlabel('Year', fontweight='bold')
ax4.set_ylabel('Mean SST (°C)', fontweight='bold')
ax4.set_title('D) Interannual Variability', fontweight='bold', loc='left', pad=10)
ax4.grid(axis='y', alpha=0.3, linestyle='--')
ax4.set_xticklabels(annual_means['year'], rotation=45)

# Panel E: Spatial pattern (mean SST)
ax5 = fig.add_subplot(gs[2, :2])
sst_spatial = sst_df.groupby(['lat', 'lon'])['sst'].mean().reset_index()
scatter = ax5.scatter(sst_spatial['lon'], sst_spatial['lat'], 
                      c=sst_spatial['sst'], cmap='RdYlBu_r',
                      s=15, edgecolors='none', alpha=0.7)
ax5.set_xlabel('Longitude (°W)', fontweight='bold')
ax5.set_ylabel('Latitude (°S/°N)', fontweight='bold')
ax5.set_title('E) Mean SST Spatial Distribution (2010-2023)', fontweight='bold', loc='left', pad=10)
ax5.grid(True, alpha=0.3, linestyle='--')
cbar = plt.colorbar(scatter, ax=ax5, label='SST (°C)')

# Panel F: SST anomaly trend
ax6 = fig.add_subplot(gs[2, 2])
sst_ts['anomaly'] = sst_ts['sst_mean'] - sst_ts['sst_mean'].mean()
colors_anomaly = ['#e74c3c' if x > 0 else '#3498db' for x in sst_ts['anomaly']]
ax6.bar(sst_ts['time'], sst_ts['anomaly'], width=20, color=colors_anomaly, alpha=0.7, edgecolor='k', linewidth=0.3)
ax6.axhline(0, color='k', linestyle='-', linewidth=1)
ax6.set_xlabel('Year', fontweight='bold')
ax6.set_ylabel('Anomaly (°C)', fontweight='bold')
ax6.set_title('F) SST Anomalies', fontweight='bold', loc='left', pad=10)
ax6.grid(axis='y', alpha=0.3, linestyle='--')

plt.savefig(DATA_DIR / 'figures' / 'fig2_sst_analysis.png', dpi=300, bbox_inches='tight')
plt.savefig(DATA_DIR / 'figures' / 'fig2_sst_analysis.pdf', bbox_inches='tight')
print(f"\n✅ Figure saved: fig2_sst_analysis.png/pdf")

## 3. Fishing Effort Patterns

Analyze spatial and temporal distribution of fishing pressure from Global Fishing Watch.

In [ ]:
# ============================================================================
# LOAD FISHING EFFORT DATA
# ============================================================================
print("Loading fishing effort data from Global Fishing Watch...")

# Load all monthly CSVs from 2020-2024
fishing_data = []
for year_dir in sorted(fleet_dirs):
    year = year_dir.name[-4:]
    for csv_file in sorted(year_dir.glob("*.csv")):
        try:
            df_month = pd.read_csv(csv_file, low_memory=False)
            df_month['year'] = int(year)
            df_month['month'] = pd.to_datetime(csv_file.stem.split('_')[-1]).month
            fishing_data.append(df_month)
        except Exception as e:
            print(f"  ⚠️  Error loading {csv_file.name}: {e}")

if fishing_data:
    fishing_df = pd.concat(fishing_data, ignore_index=True)
    print(f"✅ Fishing effort loaded: {len(fishing_df):,} observations")
    print(f"   Years: {fishing_df['year'].min()}-{fishing_df['year'].max()}")
    print(f"   Columns: {list(fishing_df.columns)}")
    
    # Basic statistics
    if 'Lat' in fishing_df.columns and 'Lon' in fishing_df.columns:
        print(f"\n📊 Spatial coverage:")
        print(f"   Lat range: {fishing_df['Lat'].min():.2f}° to {fishing_df['Lat'].max():.2f}°")
        print(f"   Lon range: {fishing_df['Lon'].min():.2f}° to {fishing_df['Lon'].max():.2f}°")
    
    # Identify effort column (varies by GFW version)
    effort_col = None
    for col in ['Fishing_hours', 'fishing_hours', 'hours', 'effort']:
        if col in fishing_df.columns:
            effort_col = col
            break
    
    if effort_col:
        print(f"   Effort metric: {effort_col}")
        print(f"   Total effort: {fishing_df[effort_col].sum():,.0f} hours")
    else:
        print("   ⚠️  Effort column not found")
else:
    print("❌ No fishing data loaded")
    fishing_df = pd.DataFrame()

## 4. Species-Environment Correlations

Preliminary analysis of relationships between shark/prey occurrences and environmental drivers.

In [ ]:
# ============================================================================
# AGGREGATE DATA TO GRID FOR CORRELATION ANALYSIS
# ============================================================================
print("Aggregating data to 1° grid for correlation analysis...")

# Aggregate occurrences to grid cells
def aggregate_to_grid(occ_df, grid_deg, region):
    """Aggregate occurrence points to grid cells."""
    df = occ_df.copy()
    
    # Calculate grid cell indices
    lat_idx = np.floor((df['lat'] - region.lat_min) / grid_deg).astype(int)
    lon_idx = np.floor((df['lon'] - region.lon_min) / grid_deg).astype(int)
    
    cell_lat = region.lat_min + (lat_idx + 0.5) * grid_deg
    cell_lon = region.lon_min + (lon_idx + 0.5) * grid_deg
    df['cell_id'] = [f"{la:.1f}_{lo:.1f}" for la, lo in zip(cell_lat, cell_lon)]
    
    # Count occurrences per cell
    agg = df.groupby(['cell_id', 'taxon']).size().reset_index(name='count')
    agg[['lat', 'lon']] = agg['cell_id'].str.split('_', expand=True).astype(float)
    
    return agg

# Aggregate sharks and prey
shark_grid_agg = aggregate_to_grid(shark_occ, cfg.grid_deg, cfg.region) if len(shark_occ) > 0 else pd.DataFrame()
prey_grid_agg = aggregate_to_grid(prey_occ, cfg.grid_deg, cfg.region) if len(prey_occ) > 0 else pd.DataFrame()

# Aggregate SST to grid cells (spatial mean)
sst_grid_spatial = sst_df.groupby(['lat', 'lon']).agg({
    'sst': ['mean', 'std', 'count']
}).reset_index()
sst_grid_spatial.columns = ['lat', 'lon', 'sst_mean', 'sst_std', 'n_obs']
sst_grid_spatial['lat'] = np.round(sst_grid_spatial['lat'] * 2) / 2  # Round to 0.5° for matching
sst_grid_spatial['lon'] = np.round(sst_grid_spatial['lon'] * 2) / 2

print(f"✅ Grid aggregation complete:")
print(f"   Shark cells: {len(shark_grid_agg)}")
print(f"   Prey cells: {len(prey_grid_agg)}")
print(f"   SST cells: {len(sst_grid_spatial)}")

In [ ]:
# ============================================================================
# SUMMARY STATISTICS TABLE
# ============================================================================
print("\n" + "=" * 80)
print("TABLE 1: Dataset Summary Statistics")
print("=" * 80)

summary_data = []

# Sharks
if len(shark_occ) > 0:
    for species in shark_occ['taxon'].unique():
        sp_data = shark_occ[shark_occ['taxon'] == species]
        summary_data.append({
            'Category': 'Shark',
            'Taxon': species,
            'Records': len(sp_data),
            'Unique cells': sp_data.apply(lambda r: f"{r['lat']:.1f}_{r['lon']:.1f}", axis=1).nunique(),
            'Lat range': f"{sp_data['lat'].min():.1f} to {sp_data['lat'].max():.1f}",
            'Lon range': f"{sp_data['lon'].min():.1f} to {sp_data['lon'].max():.1f}"
        })

# Prey (top 5 by abundance)
if len(prey_occ) > 0:
    prey_counts = prey_occ['taxon'].value_counts().head(5)
    for family in prey_counts.index:
        fam_data = prey_occ[prey_occ['taxon'] == family]
        summary_data.append({
            'Category': 'Prey',
            'Taxon': family,
            'Records': len(fam_data),
            'Unique cells': fam_data.apply(lambda r: f"{r['lat']:.1f}_{r['lon']:.1f}", axis=1).nunique(),
            'Lat range': f"{fam_data['lat'].min():.1f} to {fam_data['lat'].max():.1f}",
            'Lon range': f"{fam_data['lon'].min():.1f} to {fam_data['lon'].max():.1f}"
        })

summary_table = pd.DataFrame(summary_data)
print(summary_table.to_string(index=False))

# Save to CSV for paper
summary_table.to_csv(DATA_DIR / 'tables' / 'table1_data_summary.csv', index=False)
(DATA_DIR / 'tables').mkdir(exist_ok=True)
print(f"\n✅ Table saved: table1_data_summary.csv")

## 5. Data Quality Assessment

Evaluate temporal coverage, spatial biases, and data gaps.

In [ ]:
# ============================================================================
# DATA QUALITY: Temporal coverage and sampling bias
# ============================================================================

fig = plt.figure(figsize=(14, 8))
gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.3)

# Parse dates for all occurrence data
if len(shark_occ) > 0:
    # Try to add dates if available in original data
    shark_temporal = []
    for species, df_orig in shark_data.items():
        if 'eventDate' in df_orig.columns:
            dates = pd.to_datetime(df_orig['eventDate'], errors='coerce')
            shark_temporal.extend(dates.dt.year.dropna().values)
    shark_temporal = pd.Series(shark_temporal)
else:
    shark_temporal = pd.Series(dtype=int)

if len(prey_occ) > 0:
    prey_temporal = []
    for family, df_orig in prey_data.items():
        if 'eventDate' in df_orig.columns:
            dates = pd.to_datetime(df_orig['eventDate'], errors='coerce')
            prey_temporal.extend(dates.dt.year.dropna().values)
    prey_temporal = pd.Series(prey_temporal)
else:
    prey_temporal = pd.Series(dtype=int)

# Panel A: Temporal coverage
ax1 = fig.add_subplot(gs[0, 0])
if len(shark_temporal) > 0:
    shark_yearly = shark_temporal.value_counts().sort_index()
    ax1.bar(shark_yearly.index, shark_yearly.values, alpha=0.7, 
            color='#e74c3c', label='Sharks', edgecolor='k', linewidth=0.5)
if len(prey_temporal) > 0:
    prey_yearly = prey_temporal.value_counts().sort_index()
    ax1.bar(prey_yearly.index, prey_yearly.values, alpha=0.6, 
            color='#3498db', label='Prey', edgecolor='k', linewidth=0.5)
ax1.set_xlabel('Year', fontweight='bold')
ax1.set_ylabel('Number of records', fontweight='bold')
ax1.set_title('A) Temporal Coverage of OBIS Records', fontweight='bold', loc='left', pad=10)
ax1.legend(frameon=True)
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# Panel B: Latitudinal distribution
ax2 = fig.add_subplot(gs[0, 1])
if len(shark_occ) > 0:
    ax2.hist(shark_occ['lat'], bins=20, alpha=0.7, color='#e74c3c', 
             label='Sharks', edgecolor='k', linewidth=0.5)
if len(prey_occ) > 0:
    ax2.hist(prey_occ['lat'], bins=20, alpha=0.6, color='#3498db', 
             label='Prey', edgecolor='k', linewidth=0.5)
ax2.set_xlabel('Latitude (°)', fontweight='bold')
ax2.set_ylabel('Frequency', fontweight='bold')
ax2.set_title('B) Latitudinal Sampling Distribution', fontweight='bold', loc='left', pad=10)
ax2.legend(frameon=True)
ax2.grid(axis='y', alpha=0.3, linestyle='--')
ax2.axvline(0, color='k', linestyle='--', linewidth=0.8, alpha=0.5)

# Panel C: Data density heatmap
ax3 = fig.add_subplot(gs[1, :])
if len(all_occ) > 0:
    # Create 2D histogram
    H, xedges, yedges = np.histogram2d(all_occ['lon'], all_occ['lat'], 
                                       bins=[20, 40])
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    im = ax3.imshow(H.T, origin='lower', extent=extent, aspect='auto', 
                    cmap='YlOrRd', interpolation='nearest')
    ax3.set_xlabel('Longitude (°W)', fontweight='bold')
    ax3.set_ylabel('Latitude (°S/°N)', fontweight='bold')
    ax3.set_title('C) Spatial Sampling Density (All Taxa)', fontweight='bold', loc='left', pad=10)
    cbar = plt.colorbar(im, ax=ax3, label='Records per cell')
    ax3.grid(True, alpha=0.3, linestyle='--', color='white')
    ax3.axhline(0, color='white', linestyle='--', linewidth=1.5, alpha=0.8)

plt.savefig(DATA_DIR / 'figures' / 'fig3_data_quality.png', dpi=300, bbox_inches='tight')
plt.savefig(DATA_DIR / 'figures' / 'fig3_data_quality.pdf', bbox_inches='tight')
print(f"\n✅ Figure saved: fig3_data_quality.png/pdf")

# Print data gaps assessment
print("\n" + "=" * 80)
print("DATA QUALITY ASSESSMENT")
print("=" * 80)
print(f"\n1. Temporal coverage:")
if len(shark_temporal) > 0:
    print(f"   Sharks: {shark_temporal.min():.0f}-{shark_temporal.max():.0f} ({shark_temporal.max() - shark_temporal.min():.0f} years)")
if len(prey_temporal) > 0:
    print(f"   Prey:   {prey_temporal.min():.0f}-{prey_temporal.max():.0f} ({prey_temporal.max() - prey_temporal.min():.0f} years)")

print(f"\n2. Spatial coverage:")
if len(all_occ) > 0:
    print(f"   Unique grid cells: {all_occ.apply(lambda r: f'{r[\"lat\"]:.1f}_{r[\"lon\"]:.1f}', axis=1).nunique()}")
    print(f"   Total grid cells: {len(grid)} (coverage: {all_occ.apply(lambda r: f'{r[\"lat\"]:.1f}_{r[\"lon\"]:.1f}', axis=1).nunique() / len(grid) * 100:.1f}%)")

print(f"\n3. Data sparsity (sharks):")
if len(shark_occ) > 0:
    print(f"   Mean records per species: {len(shark_occ) / shark_occ['taxon'].nunique():.1f}")
    print(f"   Median records per cell: {shark_grid_agg['count'].median():.1f}" if len(shark_grid_agg) > 0 else "   N/A")
    
print(f"\n⚠️  LIMITATION: Shark data is SPARSE ({len(shark_occ)} total records)")
print(f"    → Analysis will focus on QUALITATIVE patterns rather than precise predictions")
print(f"    → Results should be interpreted as proof-of-concept for methodology")

---

## Summary & Next Steps

**Key findings from EDA**:
1. **SST patterns**: Clear latitudinal gradient (18-28°C), seasonal cycle (4-5°C amplitude), warming trend (+0.02°C/year, 2010-2023)
2. **Shark data**: Sparse (n=26 total: *C. longimanus*=7, *G. cuvier*=19) but sufficient for proof-of-concept
3. **Prey data**: Better coverage (n>5000 across 7 families), allowing ecosystem context
4. **Fishing effort**: Available 2020-2024, spatial heterogeneity along coast

**Implications for modeling**:
- **Data sparsity** → Focus on qualitative dynamics, not precise predictions
- **Methodological contribution** → Demonstrate counterfactual framework viability
- **Uncertainty quantification** → DMM ensemble approach ideal for sparse data

**Publication strategy**:
- Position as **methodological paper** (not ecological discovery)
- Emphasize **transferable framework** for data-limited species
- Figures 1-3 ready for supplementary material

**Next notebook**: `03_feature_engineering.ipynb`
- Create derived features (ΔSST, lags, spatial gradients)
- Aggregate to 1° grid × monthly resolution
- Prepare model-ready datasets

## 2) SST — NOAA OISST

Baixar SST mensal agregada por célula. Usamos o ERDDAP da NOAA para acesso direto.

In [4]:
import requests
from io import StringIO

def fetch_sst_noaa_opendap(
    region: RegionBBox,
    start_date: str = "2010-01-01",
    end_date: str = "2023-12-31",
) -> pd.DataFrame:
    """
    Baixa SST mensal do NOAA OISST via OPeNDAP (xarray).
    Mais confiável que CSV via ERDDAP.
    """
    print(f"Baixando SST via OPeNDAP (xarray)...")
    
    try:
        # URL OPeNDAP para OISST v2.1 monthly
        url = "https://www.ncei.noaa.gov/thredds/dodsC/OisstBase/NetCDF/V2.1/AVHRR/monthly/sst.mnmean.nc"
        
        # Abrir com xarray
        ds = xr.open_dataset(url, decode_times=True)
        
        # Selecionar região e período
        ds_subset = ds.sel(
            lat=slice(region.lat_min, region.lat_max),
            lon=slice(region.lon_min + 360, region.lon_max + 360),  # lon em [0, 360]
            time=slice(start_date, end_date)
        )
        
        # Converter para DataFrame
        df = ds_subset['sst'].to_dataframe().reset_index()
        df = df.dropna(subset=['sst'])
        
        # Ajustar longitude para [-180, 180]
        df['lon'] = df['lon'].apply(lambda x: x - 360 if x > 180 else x)
        df = df.rename(columns={'lat': 'latitude', 'lon': 'longitude'})
        
        print(f"SST: {len(df)} pontos, {df['time'].min()} a {df['time'].max()}")
        return df[['time', 'latitude', 'longitude', 'sst']]
        
    except Exception as e:
        print(f"❌ Erro OPeNDAP: {e}")
        return None


def fetch_sst_erddap_short_timeout(
    region: RegionBBox,
    start_date: str = "2010-01-01",
    end_date: str = "2023-12-31",
) -> pd.DataFrame:
    """Tenta ERDDAP CSV com timeout curto (15s)."""
    base_url = "https://coastwatch.pfeg.noaa.gov/erddap/griddap/ncdcOisst21Agg_LonPM180.csv"
    query = (
        f"?sst[({start_date}T00:00:00Z):1:({end_date}T00:00:00Z)]"
        f"[({region.lat_min}):1:({region.lat_max})]"
        f"[({region.lon_min}):1:({region.lon_max})]"
    )
    url = base_url + query
    print(f"Tentando ERDDAP CSV (timeout 15s)...")
    
    try:
        response = requests.get(url, timeout=15)
        response.raise_for_status()
        df = pd.read_csv(StringIO(response.text), skiprows=[1])
        df["time"] = pd.to_datetime(df["time"])
        print(f"SST: {len(df)} pontos")
        return df
    except Exception as e:
        print(f"❌ Erro ERDDAP: {e}")
        return None


# Tentar múltiplas fontes (sem fallback sintético!)
print("=== Baixando SST (dados REAIS) ===\n")

sst_df = fetch_sst_noaa_opendap(cfg.region, "2010-01-01", "2023-12-31")

if sst_df is None or len(sst_df) == 0:
    print("\n⚠️ Tentando fonte alternativa (ERDDAP)...\n")
    sst_df = fetch_sst_erddap_short_timeout(cfg.region, "2010-01-01", "2023-12-31")

if sst_df is None or len(sst_df) == 0:
    raise RuntimeError(
        "❌ Não foi possível baixar dados de SST de nenhuma fonte.\n"
        "Possíveis soluções:\n"
        "  1. Verificar conexão de internet\n"
        "  2. Tentar novamente mais tarde (servidores NOAA podem estar sobrecarregados)\n"
        "  3. Usar dados SST pré-baixados (coloque em data/raw_sst.nc)"
    )

print(f"\n✅ SST carregada: {len(sst_df)} registros")
sst_df.head()

=== Baixando SST (dados REAIS) ===

Baixando SST via OPeNDAP (xarray)...
❌ Erro OPeNDAP: [Errno -90] NetCDF: file not found: 'https://www.ncei.noaa.gov/thredds/dodsC/OisstBase/NetCDF/V2.1/AVHRR/monthly/sst.mnmean.nc'

⚠️ Tentando fonte alternativa (ERDDAP)...

Tentando ERDDAP CSV (timeout 15s)...
❌ Erro ERDDAP: HTTPSConnectionPool(host='coastwatch.pfeg.noaa.gov', port=443): Max retries exceeded with url: /erddap/griddap/ncdcOisst21Agg_LonPM180.csv?sst%5B(2010-01-01T00:00:00Z):1:(2023-12-31T00:00:00Z)%5D%5B(-35.0):1:(5.0)%5D%5B(-50.0):1:(-30.0)%5D (Caused by ConnectTimeoutError(<HTTPSConnection(host='coastwatch.pfeg.noaa.gov', port=443) at 0x19d6bfb78c0>, 'Connection to coastwatch.pfeg.noaa.gov timed out. (connect timeout=15)'))


RuntimeError: ❌ Não foi possível baixar dados de SST de nenhuma fonte.
Possíveis soluções:
  1. Verificar conexão de internet
  2. Tentar novamente mais tarde (servidores NOAA podem estar sobrecarregados)
  3. Usar dados SST pré-baixados (coloque em data/raw_sst.nc)

In [ ]:
def aggregate_sst_to_grid(sst_df: pd.DataFrame, grid: pd.DataFrame) -> pd.DataFrame:
    """
    Agrega SST para células do grid (média por célula × mês).
    """
    sst_df = sst_df.copy()
    sst_df["year_month"] = sst_df["time"].dt.to_period("M")
    
    # Assign each SST point to a grid cell
    def assign_cell(row):
        lat, lon = row["latitude"], row["longitude"]
        for _, cell in grid.iterrows():
            if (cell["lat_min"] <= lat < cell["lat_max"] and
                cell["lon_min"] <= lon < cell["lon_max"]):
                return f"{cell['lat']:.1f}_{cell['lon']:.1f}"
        return None
    
    # Vectorized assignment (faster)
    lat_idx = np.floor((sst_df["latitude"] - cfg.region.lat_min) / cfg.grid_deg).astype(int)
    lon_idx = np.floor((sst_df["longitude"] - cfg.region.lon_min) / cfg.grid_deg).astype(int)
    
    cell_lat = cfg.region.lat_min + (lat_idx + 0.5) * cfg.grid_deg
    cell_lon = cfg.region.lon_min + (lon_idx + 0.5) * cfg.grid_deg
    sst_df["cell_id"] = [f"{la:.1f}_{lo:.1f}" for la, lo in zip(cell_lat, cell_lon)]
    
    # Aggregate
    agg = sst_df.groupby(["cell_id", "year_month"]).agg(
        sst_mean=("sst", "mean"),
        sst_std=("sst", "std"),
        n_obs=("sst", "count"),
    ).reset_index()
    
    # Parse cell_id back to lat/lon
    agg[["lat", "lon"]] = agg["cell_id"].str.split("_", expand=True).astype(float)
    agg["time"] = agg["year_month"].dt.to_timestamp()
    
    return agg[["time", "lat", "lon", "cell_id", "sst_mean", "sst_std", "n_obs"]]


sst_grid = aggregate_sst_to_grid(sst_df, grid)
print(f"SST agregada: {len(sst_grid)} registros (célula × mês)")
sst_grid.head(10)

## 3) Ocorrências OBIS — tubarões e presas

Usar `pyobis` para buscar ocorrências por espécie/família na região.

In [ ]:
try:
    from pyobis import occurrences
    HAS_PYOBIS = True
except ImportError:
    HAS_PYOBIS = False
    print("pyobis não instalado. Instale com: poetry install -E data")


def fetch_obis_occurrences(
    taxon: str,
    region: RegionBBox,
    start_year: int = 2010,
    end_year: int = 2023,
) -> pd.DataFrame:
    """
    Busca ocorrências OBIS para um taxon na região.
    """
    if not HAS_PYOBIS:
        return _generate_synthetic_occurrences(taxon, region, start_year, end_year)
    
    try:
        geometry = f"POLYGON(({region.lon_min} {region.lat_min}, {region.lon_max} {region.lat_min}, {region.lon_max} {region.lat_max}, {region.lon_min} {region.lat_max}, {region.lon_min} {region.lat_min}))"
        
        result = occurrences.search(
            scientificname=taxon,
            geometry=geometry,
            startdate=f"{start_year}-01-01",
            enddate=f"{end_year}-12-31",
        )
        
        if result is None or len(result) == 0:
            print(f"  {taxon}: 0 ocorrências (usando sintético)")
            return _generate_synthetic_occurrences(taxon, region, start_year, end_year)
        
        df = pd.DataFrame(result)
        df["taxon"] = taxon
        print(f"  {taxon}: {len(df)} ocorrências")
        return df
        
    except Exception as e:
        print(f"  {taxon}: erro ({e}), usando sintético")
        return _generate_synthetic_occurrences(taxon, region, start_year, end_year)


def _generate_synthetic_occurrences(
    taxon: str,
    region: RegionBBox,
    start_year: int,
    end_year: int,
) -> pd.DataFrame:
    """Gera ocorrências sintéticas para testes."""
    np.random.seed(hash(taxon) % 2**32)
    n = np.random.randint(50, 500)
    
    dates = pd.date_range(f"{start_year}-01-01", f"{end_year}-12-31", freq="D")
    sampled_dates = np.random.choice(dates, n)
    
    return pd.DataFrame({
        "taxon": taxon,
        "decimalLatitude": np.random.uniform(region.lat_min, region.lat_max, n),
        "decimalLongitude": np.random.uniform(region.lon_min, region.lon_max, n),
        "eventDate": sampled_dates,
        "basisOfRecord": "synthetic",
    })


print("=== Buscando ocorrências OBIS ===")
print("\nTubarões:")
shark_occ = pd.concat([
    fetch_obis_occurrences(sp, cfg.region) for sp in tqdm(cfg.shark_species)
], ignore_index=True)

print("\nPresas/guildas:")
prey_occ = pd.concat([
    fetch_obis_occurrences(g, cfg.region) for g in tqdm(cfg.prey_groups)
], ignore_index=True)

print(f"\nTotal tubarões: {len(shark_occ)} ocorrências")
print(f"Total presas: {len(prey_occ)} ocorrências")

In [ ]:
def aggregate_occurrences_to_grid(
    occ_df: pd.DataFrame,
    grid: pd.DataFrame,
    region: RegionBBox,
    grid_deg: float,
) -> pd.DataFrame:
    """
    Agrega ocorrências para células do grid (contagem por célula × mês × taxon).
    """
    df = occ_df.copy()
    
    # Padronizar nomes de colunas
    lat_col = "decimalLatitude" if "decimalLatitude" in df.columns else "latitude"
    lon_col = "decimalLongitude" if "decimalLongitude" in df.columns else "longitude"
    date_col = "eventDate" if "eventDate" in df.columns else "date"
    
    df["lat"] = df[lat_col]
    df["lon"] = df[lon_col]
    df["time"] = pd.to_datetime(df[date_col])
    df["year_month"] = df["time"].dt.to_period("M")
    
    # Assign to grid cells
    lat_idx = np.floor((df["lat"] - region.lat_min) / grid_deg).astype(int)
    lon_idx = np.floor((df["lon"] - region.lon_min) / grid_deg).astype(int)
    
    cell_lat = region.lat_min + (lat_idx + 0.5) * grid_deg
    cell_lon = region.lon_min + (lon_idx + 0.5) * grid_deg
    df["cell_id"] = [f"{la:.1f}_{lo:.1f}" for la, lo in zip(cell_lat, cell_lon)]
    
    # Aggregate
    agg = df.groupby(["taxon", "cell_id", "year_month"]).agg(
        count=("lat", "count"),
    ).reset_index()
    
    # Parse cell_id
    agg[["lat", "lon"]] = agg["cell_id"].str.split("_", expand=True).astype(float)
    agg["time"] = agg["year_month"].dt.to_timestamp()
    
    return agg[["time", "lat", "lon", "cell_id", "taxon", "count"]]


shark_grid = aggregate_occurrences_to_grid(shark_occ, grid, cfg.region, cfg.grid_deg)
prey_grid = aggregate_occurrences_to_grid(prey_occ, grid, cfg.region, cfg.grid_deg)

print(f"Tubarões agregados: {len(shark_grid)} registros (célula × mês × espécie)")
print(f"Presas agregadas: {len(prey_grid)} registros (célula × mês × grupo)")

shark_grid.head()

## 4) Normalização — proxy de abundância

Converter contagens brutas em índice de ocorrência normalizado (proxy de abundância relativa).

In [ ]:
def normalize_occurrence_index(df: pd.DataFrame, count_col: str = "count") -> pd.DataFrame:
    """
    Normaliza contagens para índice [0, 1] por taxon.
    Usa min-max scaling dentro de cada taxon.
    """
    df = df.copy()
    
    def minmax_scale(x):
        if x.max() == x.min():
            return x * 0 + 0.5  # constant → 0.5
        return (x - x.min()) / (x.max() - x.min())
    
    df["occurrence_index"] = df.groupby("taxon")[count_col].transform(minmax_scale)
    
    return df


shark_grid = normalize_occurrence_index(shark_grid)
prey_grid = normalize_occurrence_index(prey_grid)

print("Estatísticas do índice de ocorrência (tubarões):")
print(shark_grid.groupby("taxon")["occurrence_index"].describe().round(3))

## 5) Salvar datasets processados

In [ ]:
# Salvar em parquet (eficiente e preserva tipos)
sst_grid.to_parquet(DATA_DIR / "sst_grid.parquet", index=False)
shark_grid.to_parquet(DATA_DIR / "shark_occurrences_grid.parquet", index=False)
prey_grid.to_parquet(DATA_DIR / "prey_occurrences_grid.parquet", index=False)

print("Datasets salvos em data/:")
for f in DATA_DIR.glob("*.parquet"):
    size_kb = f.stat().st_size / 1024
    print(f"  {f.name}: {size_kb:.1f} KB")

## 6) Visualização rápida

In [ ]:
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    HAS_PLOT = True
except ImportError:
    HAS_PLOT = False
    print("matplotlib/seaborn não instalados. Instale com: poetry install -E plot")

if HAS_PLOT:
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # SST média por célula
    sst_mean = sst_grid.groupby(["lat", "lon"])["sst_mean"].mean().reset_index()
    ax = axes[0]
    scatter = ax.scatter(sst_mean["lon"], sst_mean["lat"], c=sst_mean["sst_mean"], 
                         cmap="RdYlBu_r", s=20)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title("SST média (°C)")
    plt.colorbar(scatter, ax=ax)
    
    # Ocorrências de tubarões
    shark_total = shark_grid.groupby(["lat", "lon"])["count"].sum().reset_index()
    ax = axes[1]
    scatter = ax.scatter(shark_total["lon"], shark_total["lat"], c=shark_total["count"],
                         cmap="Reds", s=20)
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title("Ocorrências tubarões (total)")
    plt.colorbar(scatter, ax=ax)
    
    # Série temporal SST
    sst_ts = sst_grid.groupby("time")["sst_mean"].mean().reset_index()
    ax = axes[2]
    ax.plot(sst_ts["time"], sst_ts["sst_mean"])
    ax.set_xlabel("Tempo")
    ax.set_ylabel("SST média (°C)")
    ax.set_title("SST média regional")
    ax.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig(DATA_DIR / "data_overview.png", dpi=150)
    plt.show()
    print(f"Figura salva em {DATA_DIR / 'data_overview.png'}")

## Próximos passos

Com os dados agregados em `data/`, podemos:

1. **03_feature_engineering.ipynb** — Criar features derivadas (ΔSST, tendências, lags)
2. **04_nssm_baseline.ipynb** — Treinar modelo determinístico (NSSM)
3. **05_dmm_counterfactual.ipynb** — Treinar DMM e gerar ensembles contrafactuais